In [ ]:
import io
import tempfile
import zipfile
from os import path
from urllib import request

import geopandas as gpd
import numpy as np
import osmnx as ox
import rasterio as rio
from rasterio import features, mask
from shapely import geometry

EPSG = 21781
DEM_URL = "https://data.geo.admin.ch/ch.swisstopo.digitales-hoehenmodell_25/data.zip"
ELEV_ZONES = [0, 1000, 1500]  # lower than 1000, between 1000 and 1500, higher than 1500

In [ ]:
nominatim_query = "District de la Veveyse, Fribourg, Switzerland"
dst_filepath = "../data/processed/elev-zones.gpkg"

In [ ]:
# open the elevation data from the zip url
resp = request.urlopen(DEM_URL)
with zipfile.ZipFile(io.BytesIO(resp.read())) as zip_ref:
    with tempfile.TemporaryDirectory() as tmpdirname:
        zip_ref.extractall(tmpdirname)
        dem_filepath = path.join(tmpdirname, "DHM200.xyz")
        # create a vector layer with the elevation zones of the nominatim query extent
        with rio.open(dem_filepath) as src:
            # get the extent from nominatim
            extent_gdf = ox.geocode_to_gdf(nominatim_query).to_crs(epsg=EPSG)
            # read the elevation data for the extent
            dem_arr, dem_transform = mask.mask(src, extent_gdf.geometry, crop=True)
            # create a vector layer with the elevation zones
            zones = np.digitize(dem_arr[0], ELEV_ZONES, right=True).astype(np.uint8)
            print(zones)
            # vectorize raster features into a geo-dataframe
            zones = (
                gpd.GeoDataFrame(
                    [
                        (val, geometry.shape(geom))
                        for geom, val in features.shapes(zones, transform=dem_transform)
                        if val != 0
                    ],
                    columns=["elev-zone", "geometry"],
                    crs=extent_gdf.crs,
                )
                .dissolve(by="elev-zone")
                .rename(index={1: "<1000", 2: "1000-1500", 3: ">1500"})
            )
# dump to a file
zones.to_file(dst_filepath)